<a href="https://colab.research.google.com/github/ninjacode01/Assignment2/blob/main/ass_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Named Entity Recognition

Lets start with reading and pre-processing the training dataset


In [1]:
!git clone 'https://github.com/ninjacode01/Assignment2'

Cloning into 'Assignment2'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), 728.04 KiB | 3.59 MiB/s, done.


In [44]:
%cd Assignment2

/content/Assignment2


In [45]:
def read_file(filename):
    with open(filename, "r") as file:
        text = file.readlines()
    return text

def process_text(text):
    X = []
    Y = []
    sentenceX = []
    sentenceY = []
    for line in text:
        split = line.split(" ")
        if len(split) > 1:
            sentenceX.append(split[0].lower())
            sentenceY.append(split[1].replace("\n", ""))
        else:
            X.append(sentenceX)
            Y.append(sentenceY)
            sentenceX = []
            sentenceY = []
    return X, Y

text = read_file("data/train.txt")
X, Y = process_text(text)

testing= read_file("data/test.txt")
X_test, Y_test= process_text(testing)

In [77]:
tags={}
k=0
for i in range(len(Y)):
  for j in range(len(Y[i])):
    if tags.get(Y[i][j])==None:
      tags[Y[i][j]]= k
      k+=1
tags



{'B-NP': 0,
 'B-PP': 1,
 'I-NP': 2,
 'B-VP': 3,
 'I-VP': 4,
 'B-SBAR': 5,
 'O': 6,
 'B-ADJP': 7,
 'B-ADVP': 8,
 'I-ADVP': 9,
 'I-ADJP': 10,
 'I-SBAR': 11,
 'I-PP': 12,
 'B-PRT': 13,
 'B-LST': 14,
 'B-INTJ': 15,
 'I-INTJ': 16,
 'B-CONJP': 17,
 'I-CONJP': 18,
 '<PAD_TAG>': 19,
 'I-PRT': 20,
 'B-UCP': 21,
 'I-UCP': 22}

In [47]:
for i in range(len(X[1])):
    print(X[1][i], Y[1][i])

chancellor O
of B-PP
the B-NP
exchequer I-NP
nigel B-NP
lawson I-NP
's B-NP
restated I-NP
commitment I-NP
to B-PP
a B-NP
firm I-NP
monetary I-NP
policy I-NP
has B-VP
helped I-VP
to I-VP
prevent I-VP
a B-NP
freefall I-NP
in B-PP
sterling B-NP
over B-PP
the B-NP
past I-NP
week I-NP
. O


In [7]:
%cd /content

/content


In [ ]:
p = set()
for i in range(len(Y)):
  a= set(Y[i])
  p=p.union(a)
len(p)

22

In [8]:
import numpy as np
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## import Word2vEC embeddings

In [9]:
import gensim


In [10]:

# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

import gensim.models.keyedvectors as kv

Mounted at /content/gdrive


In [11]:
from gensim.test.utils import datapath, get_tmpfile

In [12]:
from gensim.scripts.glove2word2vec import glove2word2vec

In [13]:
model_path = datapath('/content/gdrive/MyDrive/pre trained word2vec/glove.6B.300d.txt')
output_path = tmp_file = get_tmpfile("put_word2vec.txt")
_ = glove2word2vec(model_path, tmp_file)


<ipython-input-13-4f176eac3f4d>:3: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(model_path, tmp_file)


In [14]:
model = kv.KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:

# Convert each word in each sentence to its word embedding
embeddings = []
for sentence in X:
    sentence_embeddings = []
    for word in sentence:
        if word in model:
            sentence_embeddings.append(model[word])
    embeddings.append(sentence_embeddings)

print(len(embeddings[1][1]))


300


In [15]:
print(X[1][0],embeddings[1][0])

NameError: ignored

In [ ]:
print(type(embeddings[0][1]))

<class 'numpy.ndarray'>


In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [26]:
result=model.most_similar(positive=['woman','king'], negative=['man'], topn=10)
result

[('queen', 0.6713277101516724),
 ('princess', 0.5432624816894531),
 ('throne', 0.5386103987693787),
 ('monarch', 0.5347574949264526),
 ('daughter', 0.49802514910697937),
 ('mother', 0.49564430117607117),
 ('elizabeth', 0.4832652509212494),
 ('kingdom', 0.47747090458869934),
 ('prince', 0.4668239951133728),
 ('wife', 0.46473270654678345)]

In [ ]:
# from torch.utils.data import DataLoader,Dataset
# class BIOData(Dataset):
#     def __init__(self, data):
#         self.data = data
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, index):
#         sentence, tags = self.data[index]
#         sentence_vec = torch.tensor(sentence, dtype=torch.float32)
#         tags_vec = torch.tensor(tags, dtype=torch.long)
#         return sentence_vec, tags_vec

## convert words into word embeddings in form of tensors
## convert targets into one-hot form of tensors

## And then we will make dataloaders

In [ ]:
from torch.utils.data import Dataset,DataLoader

class EmbeddingsDataset(Dataset):
    def __init__(self, embeddings_list,targets):
        self.embeddings_list = embeddings_list
        self.entities = targets

    def __len__(self):
        return len(self.embeddings_list)

    def __getitem__(self, idx):
        sample_embeddings = self.embeddings_list[idx]
        sample_targets = self.entities[idx]
        return sample_embeddings,sample_targets


In [ ]:
dataset = EmbeddingsDataset(embeddings,Y)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)


In [ ]:
for em,t in dataloader:
  print(em[0].size())
  print(t[0],t[1])
  break

## Create Model

In [ ]:

input_size = 300
hidden_size = 64
num_classes = 22
num_layers = 1
kernel_size = 3
stride = 1
padding = 1

In [ ]:
class BiLSTM_NER(nn.Module):
    def __init__(self, n_vocab, n_embed, n_hidden, n_layers, n_tags, weights):
        super(BiLSTM_NER, self).__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.embedding = nn.Embedding(n_vocab, n_embed)
        self.embedding.weight.data.copy_(torch.from_numpy(weights))
        self.lstm = nn.LSTM(n_embed, n_hidden, n_layers, bidirectional=True, dropout=0.5)
        self.fc = nn.Linear(n_hidden * 2, n_tags)
        
    def forward(self, inputs):
        embeds = self.embedding(inputs)
        lstm_out, _ = self.lstm(embeds.view(len(inputs), 1, -1))
        tag_space = self.fc(lstm_out.view(len(inputs), -1))
        tag_scores = nn.functional.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
# Instantiate the model
cnnmodel = CNNModel(300, hidden_size, num_classes, num_layers, kernel_size, stride, padding)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnnmodel.parameters(), lr=0.001)

In [ ]:
# from torch.utils.data import Dataset, DataLoader

# class BIOTaggedSentenceDataset(Dataset):
#     def __init__(self, X, Y):
#         self.X = X
#         self.Y = Y
        
#     def __len__(self):
#         return len(self.X)
    
#     def __getitem__(self, index):
#         sentence = self.X[index]
#         tags = self.Y[index]
#         sentence_vec = torch.tensor(sentence, dtype=torch.long)
#         tags_vec = torch.tensor([TAG_TO_IDX[tag] for tag in tags], dtype=torch.long)
#         return sentence_vec, tags_vec
    
# # Example training data
# X = [
#     ['John', 'likes', 'apples'],
#     ['Mary', 'hates', 'bananas']
# ]
# Y = [
#     ['B-PER', 'O', 'O'],
#     ['B-PER', 'O', 'O']
# ]

# # Create a Dataset and DataLoader
# train_dataset = BIOTaggedSentenceDataset(X, Y)
# batch_size = 2
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Train Model

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(dataloader):
        # Forward pass
        outputs = cnnmodel(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print statistics
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


## Evaluate the model using Devset for hyperparameter tuning

## Run the model finally

In [95]:
from sklearn.preprocessing import LabelEncoder
class NERDataset(Dataset):
    def __init__(self, sentences, tags, word2vec, label_encoder,max_seq_len=15):
        self.sentences = sentences
        self.tags = tags
        self.word2vec = word2vec
        self.label_encoder = label_encoder
        self.max_seq_len = max_seq_len
        

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tag = self.tags[idx]
        # Encode the sentence using word2vec embeddings
        embedded_sentence = []
        for word in sentence:
            if word in self.word2vec:
                embedding = self.word2vec[word]
            else:
                embedding = self.word2vec["the"]
            embedded_sentence.append(embedding)
        # Pad sentence embeddings
        if len(embedded_sentence) < self.max_seq_len:
            padding = [self.word2vec["pad"]] * (self.max_seq_len - len(embedded_sentence))
            embedded_sentence.extend(padding)
        else:
            embedded_sentence = embedded_sentence[:self.max_seq_len]

        # Pad tags with a special padding token
        if len(tag) < self.max_seq_len:
            padding = ["<PAD_TAG>"] * (self.max_seq_len - len(tag))
            tag.extend(padding)
        else:
            tag = tag[:self.max_seq_len]
        # Encode the tags using the label encoder
        encoded_tags = [tags[p] for p in tag]

        return torch.FloatTensor(embedded_sentence), torch.tensor(encoded_tags, dtype=torch.long)

In [72]:
label_encoder = LabelEncoder()
all_tags = set([tag for tag_list in tags for tag in tag_list])
label_encoder.fit(list(all_tags))

LabelEncoder()

In [66]:
# class NERDataset(Dataset):
#     def __init__(self, sentences, tags, word2vec_model):
#         self.sentences = sentences
#         self.tags = tags
#         self.word2vec_model = word2vec_model

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, index):
#         sentence = self.sentences[index]
#         tag = self.tags[index]

#         # Convert words to embeddings
#         sentence_embeddings = [self.word2vec_model[word] for word in sentence]

#         return torch.FloatTensor(sentence_embeddings), torch.LongTensor(tag)
# class NERDataset(Dataset):
#     def __init__(self, sentences, tags, word2vec, max_seq_len=50):
#         self.sentences = sentences
#         self.tags = tags
#         self.word2vec = word2vec
#         self.max_seq_len = max_seq_len

#     def __len__(self):
#         return len(self.sentences)

#     def __getitem__(self, idx):
#         sentence = self.sentences[idx]
#         tag=self.tags[Y[idx]]
#         sentence_embedding = []
        
#         for word in sentence:
#             if word in self.word2vec:
#                 embedding = self.word2vec[word]
#             else:
#                 embedding = self.word2vec["the"]
#             sentence_embedding.append(embedding)
        
        # # Pad sentence embeddings
        # if len(sentence_embedding) < self.max_seq_len:
        #     padding = [self.word2vec["pad"]] * (self.max_seq_len - len(sentence_embedding))
        #     sentence_embedding.extend(padding)
        # else:
        #     sentence_embedding = sentence_embedding[:self.max_seq_len]

        # # Pad tags with a special padding token
        # if len(tag) < self.max_seq_len:
        #     padding = ["<PAD_TAG>"] * (self.max_seq_len - len(tag))
        #     tag.extend(padding)
        # else:
        #     tag = tag[:self.max_seq_len]

        # return torch.FloatTensor(sentence_embedding), tag



class BiLSTMNER(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiLSTMNER, self).__init__()

        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, input):
        lstm_out, _ = self.lstm(input.view(len(input), 1, -1))
        tag_space = self.fc(lstm_out.view(len(input), -1))
        return tag_space

In [96]:
dataset = NERDataset(X,Y, model,label_encoder)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [97]:
next(iter(dataloader))

[tensor([[[-0.3966, -0.0794,  0.2449,  ..., -0.0330,  0.0562,  0.6865],
          [-0.0666,  0.7397,  0.0369,  ..., -0.0585, -0.2657, -0.2135],
          [-0.3260,  0.2583, -0.3910,  ..., -0.5825, -0.3339, -0.0575],
          ...,
          [-0.4218,  0.0659,  0.1513,  ..., -0.6822,  0.1206, -0.6496],
          [-0.1256,  0.0136,  0.1031,  ..., -0.3422, -0.0224,  0.1368],
          [-0.0305, -0.2272,  0.3878,  ..., -0.4162, -0.6488,  0.0726]]]),
 tensor([[ 0,  2,  3,  4,  4,  0,  2,  2,  1,  0,  2,  0,  2,  6, 19]])]

In [104]:
input_size = 4500
hidden_size = 128
output_size = 15
our_model = BiLSTMNER(input_size, hidden_size, output_size)

In [105]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(our_model.parameters(), lr=0.001)

In [107]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print("ahaa")
    for i, (sentence,tag) in enumerate(dataloader):
        print('omg')
        sentence = Variable(sentence)
        tag = Variable(tag)
        tag=tag.flatten()

        optimizer.zero_grad()

        outputs = our_model(sentence)
        print(outputs)
        loss = criterion(outputs, tag)
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

ahaa
omg
tensor([[ 0.2886, -0.0753,  0.0822, -0.1233, -0.0037, -0.1135,  0.1117,  0.0332,
          0.0103, -0.1060,  0.0049,  0.1197,  0.1939,  0.2082,  0.0566]],
       grad_fn=<AddmmBackward0>)


ValueError: ignored